In [26]:
# Demo for synchronization of two data directories
import os
import io
import pandas as pd
import numpy as np
import farmhash
import time
import sqlite3

In [4]:
def create_dirdf(directory):
    if not os.path.exists(directory):
        print("Error: Directory '" + directory + "' does not exist.")
        return
    
    filenames = []
    hashvalues = []
    
    for root, subdir, files in os.walk(directory):
        for name in files:
            if not name[0] == ".": # ignore hidden files
                filepath = os.path.join(root, name)
                
                # hash full file contents
                # note: spice data encoding is mixed, so read as binary
                file = str(io.open(filepath,'rb').read()) 
                filenames.append(filepath.split(directory, 1)[1])
                hashvalues.append(farmhash.hash64(file))
                
    df = pd.DataFrame(data=hashvalues, index = filenames, columns = ["Hash"])
    df.index.name = directory
    return df

start = time.time()
dir1df = create_dirdf("./testdir1")
print("elapsed time: ", time.time() - start)

dir2df = create_dirdf("./testdir2")

print(dir1df, dir2df)

elapsed time:  0.0032210350036621094
                                           Hash
./testdir1                                     
/testfile1.txt             13167233149662072294
/testfile2.txt              2116770068367243914
/testfile3.txt             10117441339441774812
/testfile4.txt               407662078023551858
/testdir1A/testfile1A.txt   1855841718642996950                                            Hash
./testdir2                                     
/testfile1.txt             13167233149662072294
/testfile2.txt              2116770068367243914
/testfile3.txt             10117441339441774812
/testfile4.txt               407662078023551858
/testdir1A/testfile1A.txt   1855841718642996950


In [30]:
conn = sqlite3.connect('./spicedb.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE testdir1_hashes (Filename TEXT)')
c.execute("ALTER TABLE {tn} ADD COLUMN '{cn}' {ct}".format(tn='testdir1_hashes', cn='Hash', ct='TEXT'))
c.execute("INSERT OR IGNORE INTO {tn} ({idf}, {cn}) VALUES ('testfn', '12312312')".format(tn='testdir1_hashes', idf='Filename', cn='Hash'))

In [31]:
create_spicedb('./testdir1')
conn.commit()
conn.close()

In [29]:
os.remove('./spicedb.sqlite')

In [23]:
def create_spicedb(directory):
    if not os.path.exists(directory):
        print("Error: Directory '" + directory + "' does not exist.")
        return
    
    for root, subdir, files in os.walk(directory):
        for name in files:
            if not name[0] == ".": # ignore hidden files
                filepath = os.path.join(root, name)
                file = str(io.open(filepath,'rb').read())
                c.execute("INSERT OR IGNORE INTO {tn} ({idf}, {cn}) VALUES ('{fn}', '{hsh}')".format(tn='testdir1_hashes', idf='Filename', cn='Hash',fn = filepath,hsh = farmhash.hash64(file) ))

In [38]:
c.execute("SELECT * FROM testdir1_hashes WHERE Filename= './testdir1/testfile1.txt'")
all_rows = c.fetchall()
print(all_rows)

[('./testdir1/testfile1.txt', '13167233149662072294')]


[]
